In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import numpy as mp
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os




In [7]:
train = 'data/train'
test = 'data/test'


num_train = 28709
num_val = 7178
batch_size = 64
num_epoch = 50



In [13]:
train_data_gen = ImageDataGenerator(rescale = 1./255)
test_data_gen = ImageDataGenerator(rescale = 1./255)

In [14]:

train_data_generator = train_data_gen.flow_from_directory(
        train,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

test_data_generator = test_data_gen.flow_from_directory(
        test,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [15]:
# Joint fine tuning [23]
#https://github.com/atulapra/Emotion-detection/
#https://dspace.library.uvic.ca/server/api/core/bitstreams/25d0a7d6-93af-4e58-a28d-a92dd8d45ca4/content#:~:text=The%20test%20results%20obtained%20show,72%25%20accurate%20for%20five%20emotions.

In [19]:
import tensorflow.keras.optimizers.legacy as legacy_optimizers

# Create the optimizer with decay using the legacy version
optimizer = legacy_optimizers.Adam(learning_rate=0.001, decay=0.01)

In [22]:
from tensorflow.keras.layers import BatchNormalization
model = tf.keras.models.Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1) ))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu' ))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu' ))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(7, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 conv2d_25 (Conv2D)          (None, 44, 44, 64)        18496     
                                                                 
 batch_normalization_23 (Ba  (None, 44, 44, 64)        256       
 tchNormalization)                                               
                                                                 
 conv2d_26 (Conv2D)          (None, 42, 42, 64)        36928     
                                                                 
 batch_normalization_24 (Ba  (None, 42, 42, 64)        256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 21, 21, 64)       

In [ ]:

#ValueError: decay is deprecated in the new Keras optimizer, please check the docstring
 #   for valid arguments, or use the legacy optimizer, e.g., tf.keras.optimizers.legacy.Adam.
    
    

In [ ]:
mode = 'train'
if mode == "train":
    model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    model_info = model.fit_generator(
            train_data_generator,
            steps_per_epoch=num_train // batch_size,
            epochs=num_epoch,
            validation_data=test_data_generator,
            validation_steps=num_val // batch_size)
    plot_model_history(model_info)
    model.save_weights('model.h5')

In [ ]:
# plots accuracy and loss curves
def plot_model_history(model_history):
    """
    Plot Accuracy and Loss curves given the model_history
    """
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    fig.savefig('plot.png')
    plt.show()


In [ ]:
mode = "display"
elif mode == "display":
    model.load_weights('model.h5')

    # prevents openCL usage and unnecessary logging messages
    cv2.ocl.setUseOpenCL(False)

    # dictionary which assigns each label an emotion (alphabetical order)
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

    # start the webcam feed
    cap = cv2.VideoCapture(0)
    while True:
        # Find haar cascade to draw bounding box around face
        ret, frame = cap.read()
        if not ret:
            break
        facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            prediction = model.predict(cropped_img)
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()